<a href="https://colab.research.google.com/github/heidyfp/Aula4/blob/main/Soporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U -q google-generativeai

In [3]:
from flask import Flask, render_template, request
import google.generativeai as genai
import numpy as np
import pandas as pd

genai.configure(api_key='YOUR_API_KEY')
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

app = Flask("Soporte_Cliente")

In [4]:
#Listando Documentos que seran buscados
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}

DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}

DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]



In [5]:
model = "models/embedding-001"
def embed_fn(title,text, model):
  return genai.embed_content(model=model,
                                      content=text,
                                      title=title,
                                      task_type = "RETRIEVAL_DOCUMENT")['embedding']


In [6]:
df = pd.DataFrame(documents)
df.columns = ["Título","Conteudo"]

In [8]:
df["Embeddings"] = df.apply(lambda row:embed_fn(row["Título"], row["Conteudo"], model),axis=1)

In [9]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                      content=consulta,
                                      task_type = "RETRIEVAL_QUERY")
  productos_escalares = np.dot(np.stack(df['Embeddings']), embedding_da_consulta["embedding"])
  idice = np.argmax(productos_escalares)
  return df.iloc[idice]['Conteudo'] # Return text from index with max value

In [10]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0,
}

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

In [11]:
model_2 = genai.GenerativeModel('gemini-1.0-pro', generation_config=generation_config,safety_settings=safety_settings)

In [12]:
chat= model_2.start_chat(history=[])

In [16]:
prompt = input('Hola, en que podemos ayudarle: ')

while prompt!= "fin":
  trecho = gerar_e_buscar_consulta(prompt, df, model)
  prompt = f'Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que no façam  parte do texto: {trecho}'
  model_2 = genai.GenerativeModel('gemini-1.0-pro', generation_config=generation_config,safety_settings=safety_settings)
  response = model_2.generate_content(prompt)
  print(response.text, '\n')
  prompt = input('Hola, en que podemos ayudarle: ')

Hola, en que podemos ayudarle: Como enciendo el googlecar
Seu Googlecar tem uma telona sensível ao toque que te dá acesso a um monte de coisas legais, tipo GPS, música e até pra controlar o ar-condicionado. Pra usar a telona, é só tocar no ícone que você quer. Por exemplo, toca no "GPS" pra saber como chegar no seu destino ou no "Música" pra tocar suas músicas preferidas. 

Hola, en que podemos ayudarle: fin


In [14]:
prompt = input('Hola, en que podemos ayudarle: ')

while prompt!= "fin":
  pregunta = prompt
  trecho = gerar_e_buscar_consulta(pregunta, df, model)
  print('Respuesta:',trecho,'\n')
  prompt = input('Hola, en que podemos ayudarle: ')

Hola, en que podemos ayudarle: como enciendo el google car
Respuesta: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs. 

Hola, en que podemos ayudarle: fin
